In [1]:
# default_exp inventory

In [2]:
#hide
!pip install -q nbdev lambdasdk s3bz

In [3]:
#hide
import pickle
KEY = ''
PW = ''
keypath = '/Users/nic/.villa-product-master-inventory'
if KEY and PW:
  with open (keypath, 'wb') as f:
    pickle.dump({
        'KEY': KEY,
        'PW': PW
    }, f)
with open(keypath, 'rb') as f:
  creden = pickle.load(f)
USER = creden['KEY']
PW = creden['PW']

# Inventory

> API details.

In [4]:
#hide
from nbdev.showdoc import *

In [5]:
#export
from botocore.config import Config
from s3bz.s3bz import S3, Requests
from lambdasdk.lambdasdk import Lambda
import bz2, json, boto3, base64, logging

In [6]:
#export
class InventorySdk:
  
  def __init__(self, stackName = 'dev', user = None, pw = None, region = 'ap-southeast-1'):
    self.stackName = stackName
    self.lambdaClient = Lambda(user =user, pw=pw, region = region)
    self.user = user; self.pw = pw; self.region = region
    
    
  def updateWithS3(self, data, 
                   inputKeyName = 'input-data-name', 
                   inputBucketName = 'input-bucket-name', 
                   functionName='update-inventory-s3-dev-manual',
                   user= None, pw= None,
                   invocationType = 'Event'):
    user = user or self.user; pw = pw or self.pw
    
    S3.save(key = inputKeyName, 
            objectToSave = data , 
            bucket = inputBucketName,
            user=user, pw=pw)
    logging.info(f'data is saved to s3, invoking ingestion function')
    
    lambdaPayload = {
        'inputBucketName': inputBucketName,
        'inputKeyName': inputKeyName
    }
    logging.info(f'input to lambda is {lambdaPayload}')
    return self.lambdaClient.invoke(functionName= functionName ,input=lambdaPayload,
                                    invocationType= invocationType )

  def querySingleProduct(self, functionName='single-product-query-dev-manual', 
                         ib_prcode= None, user=None, pw=None):
    '''query a single product'''
    input = { 'ib_prcode': ib_prcode }
    return self.lambdaClient.invoke(
        functionName = functionName, input = input )['Payload'].read()


In [7]:
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from random import randrange
from datetime import datetime
import boto3


## generate dummy data for testing

In [8]:
#Dummy Data
numberOfRows = 1000
@dataclass_json
@dataclass
class Inventory:
  ib_prcode:str
  ib_brcode:str
  ib_cf_qty:str
  new_ib_vs_stock_cv:str

sampleLargeRandomInput = [ Inventory.from_dict({
    'ib_brcode' : str(randrange(1000,1030,1)),
    'ib_prcode' : str(randrange(10000,100000,1)),
    'ib_cf_qty' : str(randrange(-10,1000,1)),
    'new_ib_vs_stock_cv' : str(randrange(-10,1000,1))
  }).to_dict() for _ in range(numberOfRows)]
sampleLargeRandomInput[0]

{'ib_prcode': '90335',
 'ib_brcode': '1002',
 'ib_cf_qty': '672',
 'new_ib_vs_stock_cv': '956'}

## Upload the batch data

In [9]:
#hide
inputBucketName= 'input-bucket-dev-manual'
functionName= 'update-inventory-s3-dev-manual'
invocationType = 'RequestResponse'

In [10]:
#example
t0 = datetime.now()
print(f'uploading {len(sampleLargeRandomInput)} items')
sdk = InventorySdk(user=USER, pw=PW)
result = sdk.updateWithS3(
    sampleLargeRandomInput,
    inputBucketName= 'input-bucket-dev-manual', 
    functionName= 'update-inventory-s3-dev-manual',
    invocationType = 'RequestResponse'
  )
dt = datetime.now()-t0
print(f'it took {dt.seconds} s')
print(result)

uploading 1000 items
it took 15 s
{"statusCode": 200, "result": {"success": 997, "failure": 0, "failureMessage": [], "timeTaken": 13}}


In [11]:
def upload(input):
  print(f'uploading {len(input)} items')
  sdk = InventorySdk(user=USER, pw=PW)
  return sdk.updateWithS3(
      input,
      inputBucketName= inputBucketName, 
      functionName= functionName,
      invocationType = invocationType,
      user=USER,
      pw = PW
    )

upload(sampleLargeRandomInput)

uploading 1000 items


'{"statusCode": 200, "result": {"success": 997, "failure": 0, "failureMessage": [], "timeTaken": 11}}'

In [12]:
from nbdev.export import *
notebook2script()

Converted index.ipynb.
Converted inventory.ipynb.
